In [12]:
!pip install simpleai numpy

In [13]:

# Mapeo de (x,y) a (r,c)
import numpy as np
mars_map = np.load('mars_map.npy')
nr, nc = mars_map.shape


$r = n_r - round(\frac{y}{cscala})$ 

$c = round(\frac{x}{cscala})$ 

Donde $n_r$ y $n_c$ son los números de renglones y columnas de la imagen o matriz de datos (en este caso 1814 y 756), y la escala indica la cantidad de metros por cada pixel (10.0174).

In [14]:
# TODO: Cambiar variables (x,y) y no se si round este bien
cscala = 10.0174

def mapeo(x, y):
    r = nr - round(y/cscala)
    c = round(x/cscala)
    return r, c

In [20]:
#-------------------------------------------------------------------------------
#    Esqueleto de PSA para el problema de ...
#-------------------------------------------------------------------------------

from simpleai.search import SearchProblem, depth_first, breadth_first, uniform_cost, astar
from simpleai.search.viewers import BaseViewer, ConsoleViewer, WebViewer

#-------------------------------------------------------------------------------
#   Definición del problema
#-------------------------------------------------------------------------------

class Problem(SearchProblem):
    """
        Clase que es usada para crear el objeto problema. Los estados son representados con
        una estructura en Python que guarde la información descrita en la formulación del PSA.
    """

    def __init__(self, origen, destino, map):
        """ Constructor de la clase. Inicializa el problema de acuerdo un conjunto de
            argumentos.

            origen: ([row inicial, col inicial]) 

        """

        # Construye el estado inicial a partir de los argumentos especificados de acuerdo
        # a lo definido en la formulación del PSA.

        # Define las acciones posibles que puede ejecutar el agente
        self.moves = [[1,0], [-1,0], [0,1], [0,-1]] # Abajo, Arriba, Derecha, Izquierda

        # Define el mapa de marte
        self.map = map 
        # Define el tamaño del mapa en filas
        self.nr = map.shape[0]
        # Define el tamaño del mapa en columnas
        self.nc = map.shape[1]
        
        # Define el estado inicial (obligatorio).
        # Posicion inicial
        origen = (origen[0], origen[1]) 

        # ll# Llama al constructor de su superclase SearchProblem (start = estado inicial).
        SearchProblem.__init__(self, origen)      # Starting position
        self.position = (2850, 6400)
        
        # Define el estado meta (opcional).
        # Posicion final
        self.goal_state = (destino[0], destino[1])
        
    def actions(self, state):
        """
            Regresa una lista con las acciones legales del agente.

            state: [x, y] Estado actual del agente.
        """

        # Determina las acciones legales según el estado recibido y las precondiciones
        # de cada acción posible, para luego colocarlas en una lista de python

        acciones = []

        # Verifica si la acción es legal
        for move in self.moves:
            # Calcula el siguiente estado
            next_state = (state[0] + move[0], state[1] + move[1])
            # Verifica si la acción esta dentro de los limites y el valor no es -1
            if (next_state[0] >= 0 and next_state[0] < self.nr and next_state[1] >= 0 and next_state[1] < self.nc and self.map[next_state[0], next_state[1]] != -1 and abs(self.map[next_state[0], next_state[1]] - self.map[state[0], state[1]]) < 0.25 and self.map[next_state[0], next_state[1]] != -1):
                acciones.append(move)

        return acciones

    def result(self, state, action):                                   
        ''''
            Regresa el nuevo estado al ejecutar una acción                   
            state: ([row, col) Lista con la posición actual del rover.
            action: [delta_row, delta_col] Lista de enteros indicando el movimiento del rover en los ejes x e y.
        '''              
        # Implementa el modelo de transición para determinar el nuevo estado a par 
        # de aplicar la acción al estado especifica                
        
        # Calcula el nuevo estado
        new_state = (state[0] + action[0], state[1] + action[1])

        return new_state
        

    def is_goal(self, state):
        """
            Determina si se ha llegado a un estado meta.

            state: Estado a ser evaluado.
        """

        # efectúa la prueba de meta para determinar si se ha llegado a un
        # estado deseado

        if(self.goal_state == state):
            return True


    def cost(self, state, action, state2):
        """
            Regresa el costo de ejecutar una acción.

            state: ...
            action: ...
            state2: ...
        """

        # regresa un número que representa el costo de ejecutar action sobre
        # el estado state para producir el estado state2
        
        # Calcula el costo de la acción, en este acaso es 1 para todos los movimientos
        return 1


    # def heuristic(self, state):
    #     """
    #         Regresa valor de la heurística para algoritmos de búsqueda con información.

    #         state: ...
    #     """

    #     # Estima y regresa la distancia desde el estado dado a su meta más cercana.

    #     return ...

# Despliega la secuencia de estados y acciones obtenidas como resultado
def display(result):
    if result is not None:
        for i, (action, state) in enumerate(result.path()):
            if action == None:
                print('Estado inicial')
            elif i == len(result.path()) - 1:
                print(i,'- Después de la accion', action)
                print('¡Meta lograda con costo =', result.cost,'!')
            else:
                print(i,'- Después de la accion', action)

            print('  ', state)
    else:
        print('Falla: No se pudo resolver el problema')



In [21]:
#-------------------------------------------------------------------------------
#   Solución del problema con diferentes métodos
#-------------------------------------------------------------------------------



test_cases = [[mapeo(2850, 6400), mapeo(3150, 6800), mars_map]]

# posibles expectadores para las búsquedas
my_viewer = None
#my_viewer = BaseViewer()       # Solo estadísticas
#my_viewer = ConsoleViewer()    # Texto en la consola
#my_viewer = WebViewer()        # Abrir en un browser en la liga http://localhost:8000

# Crea PSAs y los resuelve usando una estrategia de búsqueda mediante
# un algoritmo seleccionado.

# resuelve el problema utilizando búsqueda de árbol con el algoritmo de
# primero en anchura
result = breadth_first(Problem(test_cases[0][0], test_cases[0][1], test_cases[0][2]), graph_search=True, viewer=my_viewer)
print()
print('>> Búsqueda Primero en Anchura <<')
display(result)

# despliega las estadísticas de búsqueda si no se seleccionó un espectador
if my_viewer != None:
    print('Estadisticas:')
    print(my_viewer.stats)

# # resuelve el problema utilizando búsqueda de grafo con el algoritmo de
# # primero en profundidad
# result = depth_first(Problem(args...), graph_search=True, viewer=my_viewer)
# print()
# print('>> Búsqueda Primero en Profundidad <<')
# display(result)

# # despliega las estadísticas de búsqueda si no se seleccionó un espectador
# if my_viewer != None:
#     print('Estadisticas:')
#     print(my_viewer.stats)

# # resuelve el problema utilizando búsqueda de grafo con el algoritmo de
# # A*. Ojo: equiere tener difinida la heurística
# result = astar(Problem(args...), graph_search=True, viewer=my_viewer)
# print()
# print('>> Búsqueda A* <<')
# display(result)

# # despliega las estadísticas de búsqueda si no se seleccionó un espectador
# if my_viewer != None:
#     print('Estadisticas:')
#     print(my_viewer.stats)

#-------------------------------------------------------------------------------
#   Fin del archivo
#-------------------------------------------------------------------------------(


>> Búsqueda Primero en Anchura <<
Falla: No se pudo resolver el problema
